In [33]:
import sys
sys.path.insert(0, '..')
import alpha_vantage
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from operator import methodcaller
import requests
import datetime
import yaml
import os
import csv
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import traceback
import sys
pd.set_option('display.max_columns', None)

In [2]:
tsdaily_dfs, pe_ratio_dfs = {}, {}
symbols = ['AAPL','AXP','GOOG','META','NVDA','AMZN','PM','SBUX','DIS','MSFT','NFLX','ICE','PLTR','SPOT','NKE','WMT','JNJ','SHOP','TSLA',
           'MA','XOM','V','HD','AMD','PEP','MCD','ABT','IBM','UPS','MO','STZ']

In [3]:
# Read CSVs
# for symbol in symbols:
for symbol in [i[:-4] for i in os.listdir('../tsdaily_dfs')]:
    tsdaily_df = pd.read_csv(f'../tsdaily_dfs/{symbol}.csv',index_col=0)
    tsdaily_df.index = pd.to_datetime(tsdaily_df.index)
    tsdaily_dfs[symbol] = tsdaily_df

for symbol in [i[:-4] for i in os.listdir('../pe_ratio_dfs')]:
    pe_ratio_df = pd.read_csv(f'../pe_ratio_dfs/{symbol}.csv',index_col=0)
    pe_ratio_df.index = pd.to_datetime(pe_ratio_df.index)
    for col in pe_ratio_df.columns[:1]:
        pe_ratio_df[col] = pd.to_datetime(pe_ratio_df[col], errors='coerce')
    pe_ratio_dfs[symbol] = pe_ratio_df

In [4]:
for symbol in symbols:
  if symbol not in tsdaily_dfs.keys() or symbol not in pe_ratio_dfs.keys():
    try:
      tsdaily_dfs[symbol], pe_ratio_dfs[symbol] = alpha_vantage.generate_dfs(symbol)
    except KeyError:
      print(f'Failed at {alpha_vantage.get_prev_api_key()}')
      print(traceback.format_exc())
      break

In [ ]:
# To CSVs
# for symbol in symbols:
for symbol in tsdaily_dfs.keys():
    tsdaily_dfs[symbol].to_csv(f'../tsdaily_dfs/{symbol}.csv')
for symbol in pe_ratio_dfs.keys():
    pe_ratio_dfs[symbol].to_csv(f'../pe_ratio_dfs/{symbol}.csv')

In [7]:
# Read CSV
overview_df = pd.read_csv('../summary_info/company_overview.csv', index_col=0)

In [16]:
# summary_info = {}
summary_info = overview_df.transpose().to_dict()
for name in tsdaily_dfs.keys():
    # print(name)
    if name in summary_info.keys():
        continue
    info = alpha_vantage.pull_basic_data(name)
    if len(info.keys()) <= 2:
        print(name,info)
        break
    summary_info[name] = alpha_vantage.pull_basic_data(name)

GOOG {'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


In [34]:
overview_df.head(2)

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
V,V,Common Stock,Visa Inc. Class A,Visa Inc. is an American multinational financi...,1403161,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC","P.O. BOX 8999, SAN FRANCISCO, CA, US",September,2023-09-30,530196267000,2.287000e+10,31.17,1.603,18.310,1.87,0.0081,8.33,15.95,0.529,0.660,0.156,0.465,32653001000,28567000000,8.33,0.217,0.106,283.68,31.17,26.04,16.240,13.69,16.25,23.45,0.951,263.25,207.52,250.92,238.18,1582520000,2023-12-01,2023-11-08
MO,MO,Common Stock,Altria Group,"Altria Group, Inc. (previously known as Philip...",764180,NYSE,USD,USA,MANUFACTURING,CIGARETTES,"6601 WEST BROAD STREET, RICHMOND, VA, US",December,2023-09-30,73434350000,1.252600e+10,8.44,6.310,-1.925,3.80,0.0947,4.92,11.53,0.426,0.593,0.218,0.000,20561000000,14183000000,4.92,8.920,-0.025,46.09,8.44,8.31,3.571,27.3,4.707,7.93,0.669,44.96,38.17,41.04,43.61,1768650000,2024-01-10,2023-12-20


In [32]:
overview_df.columns

Index(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange',
       'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd',
       'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio',
       'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS',
       'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM',
       'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM',
       'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY',
       'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE',
       'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue',
       'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage',
       '200DayMovingAverage', 'SharesOutstanding', 'DividendDate',
       'ExDividendDate'],
      dtype='object')

In [39]:
bollinger_returns= {}
net_change = {}
last_price = {}
for symbol in tsdaily_dfs.keys():
  bollinger_returns[symbol] = alpha_vantage.generate_bollinger_returns(alpha_vantage.create_buy_sell_pairs(tsdaily_dfs, symbol, start_date = '2021-01-01'))
  net_change[symbol] = tsdaily_dfs[symbol].iloc[-1,3] - tsdaily_dfs[symbol].iloc[0,3]
  last_price[symbol] = tsdaily_dfs[symbol].iloc[-1,3]

metrics = pd.DataFrame([bollinger_returns,net_change,last_price],).transpose().rename(columns={0:'bollinger_returns',1:'net_change',2:'last_price'})
metrics['net_over_change'] = metrics['bollinger_returns']/metrics['net_change'].abs()*100
metrics['net_over_price'] = metrics['bollinger_returns']/metrics['last_price']*100
metrics = metrics.sort_values(by='net_over_price',ascending=False)
metrics['Symbol'] = metrics.index
metrics = metrics.iloc[:,[-1]+list(range(len(metrics.columns)-1))]

In [40]:
pd.merge(metrics,overview_df[['Symbol','Sector','Industry','Beta']],how='left',on='Symbol')

,Symbol,bollinger_returns,net_change,last_price,net_over_change,net_over_price,Sector,Industry,Beta
0,GOOG,477.40,-417.0200,141.44,114.478922,337.528281,NaN,NaN,NaN
1,MA,179.02,372.7700,418.77,48.024251,42.749003,NaN,NaN,NaN
2,XOM,30.60,28.4700,103.22,107.481560,29.645418,ENERGY & TRANSPORTATION,PETROLEUM REFINING,0.960
3,PLTR,4.25,6.7500,16.25,62.962963,26.153846,NaN,NaN,NaN
4,MSFT,95.89,281.7000,374.07,34.039759,25.634240,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,0.876
5,V,62.78,201.4800,257.98,31.159420,24.335220,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",0.951
6,MCD,71.06,252.7000,294.39,28.120301,24.138048,TRADE & SERVICES,RETAIL-EATING PLACES,0.708
7,PEP,35.13,139.3900,172.95,25.202669,20.312229,MANUFACTURING,BEVERAGES,0.537
8,AXP,36.17,38.0600,187.06,95.034157,19.336042,FINANCE,FINANCE SERVICES,1.216
9,AAPL,33.18,115.5300,193.15,28.719813,17.178359,NaN,NaN,NaN
